In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

In [2]:
dataset1 = pd.read_csv('dataset1.csv')
dataset1.label.replace(-1,0,inplace=True)
dataset2 = pd.read_csv('dataset2.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('dataset3.csv')

In [3]:
print(dataset3.shape,dataset2.shape,dataset2.shape)

(112803, 57) (257126, 56) (257126, 56)


In [4]:
pd.set_option('display.max_columns',200)   
dataset1.head()

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1832624,0.900000,0.0,29,16,200.0,20.0,1,14962.0,14962.0,14962.0,-1.0,10.0,1.209798,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,0,1,0,0,0
1,163606,0.850000,10.0,21,8,200.0,30.0,1,787.0,787.0,787.0,-1.0,10.0,2.439644,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,1,0,0,0,0
2,4061024,0.900000,10.0,26,13,200.0,20.0,1,14962.0,14962.0,14962.0,-1.0,10.0,1.209798,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,1,0,0,0,0,0,0
3,106443,0.833333,NaN,29,16,30.0,5.0,1,6553.0,6553.0,6553.0,-1.0,10.0,0.664123,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,0,1,0,0,0
4,114747,0.850000,9.0,26,13,200.0,30.0,1,787.0,787.0,787.0,-1.0,10.0,2.439644,1.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,11.0,11.0,11.0,13.4,1.0,25.0,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,1,0,0,0,0,0,0


In [5]:
dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset3.drop_duplicates(inplace=True)

In [6]:
#把前面两个数据集整合在一起
dataset12 = pd.concat([dataset1,dataset2],axis=0)

In [7]:
dataset1_y = dataset1.label
dataset1_x = dataset1.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)  
# 'day_gap_before','day_gap_after' cause overfitting, 0.77

In [8]:
dataset2_y = dataset2.label
dataset2_x = dataset2.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)

In [9]:
dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)

In [10]:
dataset3_preds = dataset3[['user_id','coupon_id','date_received']]
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received','day_gap_before','day_gap_after'],axis=1)


In [11]:
print(dataset1_x.shape,dataset2_x.shape,dataset3_x.shape)

(134416, 52) (252481, 52) (112803, 52)


In [12]:
dataset1 = xgb.DMatrix(dataset1_x,label=dataset1_y)
dataset2 = xgb.DMatrix(dataset2_x,label=dataset2_y)
dataset12 = xgb.DMatrix(dataset12_x,label=dataset12_y)
dataset3 = xgb.DMatrix(dataset3_x)

E:\anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
E:\anaconda\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [13]:
params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }

In [14]:
#train on dataset1, evaluate on dataset2
watchlist = [(dataset1,'train'),(dataset2,'val')]
model = xgb.train(params,dataset1,num_boost_round=3500,evals=watchlist,early_stopping_rounds=300)


[0]	train-auc:0.84695	val-auc:0.765507
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 300 rounds.
[1]	train-auc:0.851093	val-auc:0.772718
[2]	train-auc:0.856271	val-auc:0.781606
[3]	train-auc:0.858132	val-auc:0.790914
[4]	train-auc:0.85758	val-auc:0.792068
[5]	train-auc:0.858273	val-auc:0.792323
[6]	train-auc:0.859502	val-auc:0.796585
[7]	train-auc:0.859914	val-auc:0.795972
[8]	train-auc:0.860529	val-auc:0.797983
[9]	train-auc:0.860795	val-auc:0.798096
[10]	train-auc:0.861083	val-auc:0.798646
[11]	train-auc:0.861123	val-auc:0.797987
[12]	train-auc:0.861651	val-auc:0.797576
[13]	train-auc:0.861607	val-auc:0.796031
[14]	train-auc:0.861851	val-auc:0.797202
[15]	train-auc:0.862344	val-auc:0.797233
[16]	train-auc:0.863028	val-auc:0.797532
[17]	train-auc:0.863119	val-auc:0.796125
[18]	train-auc:0.863791	val-auc:0.794006
[19]	train-auc:0.86405	val-auc:0.795043
[20]	train-auc:0.864078	val-auc:0.795263
[21]	train-a

In [15]:
watchlist = [(dataset12,'train')]
model = xgb.train(params,dataset12,num_boost_round=3500,evals=watchlist)

[0]	train-auc:0.822773
[1]	train-auc:0.837885
[2]	train-auc:0.839082
[3]	train-auc:0.838534
[4]	train-auc:0.838964
[5]	train-auc:0.839787
[6]	train-auc:0.839812
[7]	train-auc:0.840054
[8]	train-auc:0.841698
[9]	train-auc:0.841866
[10]	train-auc:0.842235
[11]	train-auc:0.842546
[12]	train-auc:0.84269
[13]	train-auc:0.842214
[14]	train-auc:0.843521
[15]	train-auc:0.843062
[16]	train-auc:0.844094
[17]	train-auc:0.844274
[18]	train-auc:0.844032
[19]	train-auc:0.843977
[20]	train-auc:0.844182
[21]	train-auc:0.844369
[22]	train-auc:0.844317
[23]	train-auc:0.844352
[24]	train-auc:0.844921
[25]	train-auc:0.8452
[26]	train-auc:0.84555
[27]	train-auc:0.845336
[28]	train-auc:0.845238
[29]	train-auc:0.845297
[30]	train-auc:0.845327
[31]	train-auc:0.845982
[32]	train-auc:0.845825
[33]	train-auc:0.84572
[34]	train-auc:0.845847
[35]	train-auc:0.845942
[36]	train-auc:0.846259
[37]	train-auc:0.846971
[38]	train-auc:0.846945
[39]	train-auc:0.846828
[40]	train-auc:0.846754
[41]	train-auc:0.846822
[42]	tr

In [16]:
dataset3_preds['label'] = model.predict(dataset3)
dataset3_preds.label=np.array(dataset3_preds.label).reshape(-1,1)
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
dataset3_preds.label=dataset3_preds[['label']].apply(max_min_scaler)
dataset3_preds

,user_id,coupon_id,date_received,label
0,4129537,9983,20160712,0.182129
1,6949378,3429,20160706,0.422749
2,2166529,6928,20160727,0.203614
3,2166529,1808,20160727,0.203614
4,6172162,6500,20160708,0.252302
5,4005121,9983,20160706,0.345342
6,4347394,9983,20160716,0.235206
7,3094273,13602,20160727,0.286936
8,5139970,9983,20160729,0.241848
9,3237121,13602,20160703,0.267786


In [17]:
dataset3_preds.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds.to_csv("xgb_preds.csv",index=None,header=None)
dataset3_preds.head()

,user_id,coupon_id,date_received,label
58111,1302487,3,20160709,0.217546
4078,5636038,3,20160723,0.272663
100367,2735138,3,20160701,0.281547
25100,1647633,3,20160710,0.302796
88774,5155319,3,20160703,0.310949


In [64]:
#predict test set
dataset3_preds['label'] = model.predict(dataset3)
#dataset3_preds.label = MinMaxScaler().fit_transform(dataset3_preds.label.reshape(-1, 1))
#dataset3_preds.label = MinMaxScaler().fit_transform(dataset3_preds.label)

dataset3_preds.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds.to_csv("xgb_preds.csv",index=None,header=None)
#print(dataset3_preds.describe())
dataset3_preds.head()

,user_id,coupon_id,date_received,label
4078,5636038,3,20160723,0.459747
100367,2735138,3,20160701,0.459747
55059,5244608,3,20160720,0.459747
25100,1647633,3,20160710,0.459747
97952,1368563,3,20160722,0.459747


In [18]:
#save feature score
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('xgb_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)